In [ ]:
%pip install torch==2.1.0 transformers peft trl accelerate bitsandbytes datasets

In [ ]:
%pip install huggingface_hub

In [ ]:
from huggingface_hub import login

login(token="")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset
import json

In [ ]:
data_file="./data.jsonl"
model_name="mistralai/Mistral-7B-v0.1"
output_dir="results"

In [ ]:
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]
    return Dataset.from_list(data)

In [ ]:
dataset = load_data(data_file)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
)
model.config.use_cache = False
model = prepare_model_for_kbit_training(model)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

In [ ]:
def tokenize_function(examples, tokenizer):
    texts = [f"{p}{c}" for p, c in zip(examples["prompt"], examples["completion"])]
    return tokenizer(texts, truncation=True, padding="max_length", max_length=512)

In [ ]:
tokenized_dataset = dataset.map(lambda examples: tokenize_function(examples, tokenizer), batched=True, remove_columns=["prompt", "completion"])

In [ ]:
training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=3,  
        logging_dir=f"results/logs",
        logging_steps=10,
        save_steps=500,
        save_total_limit=2,
        fp16=False,  
        bf16=True,
        optim="paged_adamw_8bit",
        report_to="none",
        remove_unused_columns=False, 
    )

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=model.peft_config["default"], 
    dataset_text_field="text", 
    tokenizer=tokenizer,
    args=training_args,
    max_seq_length=512,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)